# Run novoBreak on Germline call
## Introduction
I need synthesize the NGS reads first. 

Note that the benchmark test result is from the file ../../test/2x250_bwa_germline2/output/compare_and_annotate/intrachromosomal_SVs_AJ_2x250_son

And the reference genome is: ../../../GIAB/data/refGenomes/genome.fasta
and bam files: inDir: '../../../GIAB/data/bam
as specified in the file: ../../test/2x250_bwa_germline2/config.yaml


In [1]:
wgsim
which wgsim


Program: wgsim (short read simulator)
Version: 0.3.2
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   wgsim [options] <in.ref.fa> <out.read1.fq> <out.read2.fq>

Options: -e FLOAT      base error rate [0.020]
         -d INT        outer distance between the two ends [500]
         -s INT        standard deviation [50]
         -N INT        number of read pairs [1000000]
         -1 INT        length of the first read [70]
         -2 INT        length of the second read [70]
         -r FLOAT      rate of mutations [0.0010]
         -R FLOAT      fraction of indels [0.15]
         -X FLOAT      probability an indel is extended [0.30]
         -S INT        seed for random generator [0, use the current time]
         -A FLOAT      discard if the fraction of ambiguous bases higher than FLOAT [0.05]
         -h            haplotype mode

~/miniconda3/bin/wgsim


In [2]:
# double check the reference genome 
cat ../../../GIAB/data/refGenomes/genome.fasta.fai

1	249250621	56	60	61
10	135534747	253404913	60	61
11	135006516	391198631	60	61
12	133851895	528455314	60	61
13	115169878	664538132	60	61
14	107349540	781627566	60	61
15	102531392	890766323	60	61
16	90354753	995006629	60	61
17	81195210	1086867352	60	61
18	78077248	1169415873	60	61
19	59128983	1248794466	60	61
2	243199373	1308908989	60	61
20	63025520	1556161742	60	61
21	48129895	1620237745	60	61
22	51304566	1669169862	60	61
3	198022430	1721329561	60	61
4	191154276	1922652421	60	61
5	180915260	2116992658	60	61
6	171115067	2300923229	60	61
7	159138663	2474890270	60	61
8	146364022	2636681301	60	61
9	141213431	2785484780	60	61
MT	16569	2929051823	60	61
X	155270560	2929068725	60	61
Y	59373566	3086927189	60	61
GL000192.1	547496	3147290388	60	61
GL000225.1	211173	3147847082	60	61
GL000194.1	191469	3148061848	60	61
GL000193.1	189789	3148256582	60	61
GL000200.1	187035	3148449608	60	61
GL000222.1	186861	3148639834	60	61
GL000212.1	186858	3148829883	60	61
GL000195.1	182896	3149019929	60	61
GL000223

In [15]:
module load bgzip
cd /mnt/nfs/gigantor/ifs/DCEG/Projects/CoherentLogic/SV/mocca-new-callers/novoBreak_test/GIAB
# 2x100bp x 1M reads = 0.2G
# 200x300M = 60G so we need to choose -N 30M  
# default -N 1000000 is not enough
# wgsim -e 0.001 -1 100 -2 100 -r 0 ~/CL/GIAB/data/refGenomes/genome.fasta normal.read1.fq normal.read2.fq
wgsim -e 0.001 -1 100 -2 100 -N 300000000 -r 0 ~/CL/GIAB/data/refGenomes/genome.fasta >(bgzip -c > normal.read1.fq.gz)  >(bgzip -c > normal.read2.fq.gz)

[wgsim] seed = 1601324593
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 84 sequences, total length: 3101804739


In [17]:
pwd

/mnt/nfs/gigantor/ifs/DCEG/Projects/CoherentLogic/SV/mocca-new-callers/novoBreak_test/GIAB


In [18]:
# bwa align them
module load bwa
bwa mem -t 8 -R "@RG\tID:wgsim_normal\tLB:wgsim_normal\tSM:wgsim_normal\tPL:ILLUMINA\tPI:500" ~/CL/GIAB/data/refGenomes/genome.fasta normal.read1.fq.gz normal.read2.fq.gz | samtools sort -@8 -o normal.bam -
# samtools view -Sb - > normal.bam
# | samtools sort -@8 -o normal.bam -

[M::main_mem] read 800000 sequences (80000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 354326, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (465, 499, 533)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (329, 669)
[M::mem_pestat] mean and std.dev: (498.97, 49.73)
[M::mem_pestat] low and high boundaries for proper pairs: (261, 737)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 800000 reads in 236.441 CPU sec, 29.663 real sec
[M::main_mem] read 800000 sequences (80000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 353988, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for 

## Run on novoBreak

In [ ]:
export PATH=/home/zhuw10/CL/SV/mocca-new-callers/tools/nb_distribution/:$PATH

cd /home/zhuw10/CL/SV/mocca-new-callers

run_novoBreak.sh ./tools/nb_distribution ../refGenomes/Homo_sapiens_assembly19.fasta ../../GIAB/data/bam/HG002_BWA_2x250.bam  novoBreak_test/GIAB/normal.bam  16 novoBreak_test/GIAB/nb_output

[Fri Sep 11 11:09:32 2020]
Building reference kmer...
Calculating number of paired reads ...
[fillin_bitvec] processed  879759728 reads
There are 4566546664 reference kmers and 5123741698 non-reference kmers
Finished reference kmer building
[Fri Sep 11 14:05:30 2020]
Freed reference hash and begin building reads hash table...
[Fri Sep 11 14:05:30 2020]
[main_samview] region "NC_007605:17750-18750" specifies an unknown reference name. Continue anyway.
[main_samview] region "NC_007605:17750-18750" specifies an unknown reference name. Continue anyway.
[main_samview] region "NC_007605:96179-97179" specifies an unknown reference name. Continue anyway.
[main_samview] region "NC_007605:96179-97179" specifies an unknown reference name. Continue anyway.
[build_readshash] processed  879759728 reads
[Fri Sep 11 15:34:07 2020]
 65419153 kmers loaded

[Fri Sep 11 15:34:07 2020]
Calculating kmers from control...
[cal_ctrl_kmers] processed   60000000 reads
[Fri Sep 11 15:48:17 2020]
Loading sequences

In [ ]:
# resume
# it is running on perl $nbbin/infer_bp_v4.pl $file $tumor_bam $normal_bam > $file.sp.vcf &qq
# under group_reads
# perl /mnt/nfs/gigantor/ifs/DCEG/Projects/CoherentLogic/SV/mocca-new-callers/tools/nb_distribution/infer_bp_v4.pl xaj /mnt/nfs/gigantor/ifs/DCEG/Projects/CoherentLogic/GIAB/data/bam/HG002_BWA_2x250.bam /mnt/nfs/gigantor/ifs/DCEG/Projects/CoherentLogic/SV/mocca-new-callers/novoBreak_test/GIAB/normal.bam
# it will be further processed by  tools/nb_distribution/filter_sv_icgc.pl 
# it is running on infer_bp_v4.pl
run_novoBreak.sh ./tools/nb_distribution ../refGenomes/Homo_sapiens_assembly19.fasta ../../GIAB/data/bam/HG002_BWA_2x250.bam  novoBreak_test/GIAB/normal.bam  16 novoBreak_test/GIAB/nb_output

In [20]:
tail -n 10 novoBreak_test/GIAB/nb_output/novoBreak.pass.flt.vcf

Y	58990414	N	.	<TRA>	5	PASS	PRECISE;CT=5to3;CIPOS=-10,10;CIEND=-10,10;SOMATIC;CONSENSUS=TGTCTGGGCTCTGCCTACAGGGTCATTTTAACATAGCACTGCACTGATCACCTAGATAATGTAACTCTTGTATAGGCTTTGCCGAAAGAGGCATTGAGACATATCTCTGCACTTATCACCGAGGTGATACAACTCTTGTCTGGGATCTGCCTGCAGGAGACATTTTTACATATCTCTGAAATGATCAACGAGGTGATGTTACTCTTGTCTAGGCTCTGCCTACTGGAGACATTGTGACTTATCTCTGCACTGATCACCCAGGTGATGTAACTCTTGTCTAGAGTCTGGCCACAGGGACAGAGTGACATATATCTGCACTGATCACACAGGTGATGTACTTCTTGTAAAGTCTTTGCCTACAGGGGGTGTTGTGACATATCTCTGCACTGATCTCTGAGGGGAGGTAACTCTTGTCTAGTCTCTGTCTA;SVTYPE=TRA;CHR2=GL000225.1;END=83402;SVLEN=0	GT	./.	E11522	contig8	size428	2	cov1.17	111	12	56.1666666666667	11	60	6	0	0	0	0	93	1	0	0	0	2	0	0	0	0	133	0	84	0
Y	58991731	N	.	<TRA>	36	PASS	PRECISE;CT=3to5;CIPOS=-10,10;CIEND=-10,10;SOMATIC;CONSENSUS=TGACATATCTCTGCACTGATCACCCAGGTGATATAACTCTTGTCTAGGATCTGCCTAAATGGACTTTGTGACAGAACCCTGTACTGATCATCCAGGTGATGGGGCTTTTGTCTAGGCTCTTCCTACGGGGGCATTCTGACTTATTTCTGCACTGATCACCCAGGTGACGGACTCTTGTCTTGCATCTGCCTATGGGGACATTGTGACATATCTCTGCACTGATCACCCAGGTGATGG

## Compare to GIAB truth vcf file

In [32]:
COMPUTE_VCF=novoBreak_test/GIAB/nb_output/novoBreak.pass.flt.vcf 
COMPUTE_BED=novoBreak_test/GIAB/nb_output/novoBreak.pass.bed
ISEC_BED=novoBreak_test/GIAB/nb_output/novoBreak.isec.bed
TRUTH_PAD=~/CL/SV/mocca-bench/data/truth_giab_padded.bed

In [26]:
grep -c -v "^#" $COMPUTE_VCF

100098


In [27]:
./scripts/novoBreak2bed.pl $COMPUTE_VCF  > $COMPUTE_BED
wc -l $COMPUTE_BED

16370 novoBreak_test/GIAB/nb_output/novoBreak.pass.bed


In [28]:
cut -f 5 $COMPUTE_BED | sort | uniq -c

   6453 DEL
   5048 DUP
   4869 INV


In [33]:
bedtools intersect -a $TRUTH_PAD -b $COMPUTE_BED -wa -wb -f 0.7 -r > $ISEC_BED

wc -l $TRUTH_PAD
wc -l $COMPUTE_BED 
cut -f 4 $ISEC_BED | sort -u |wc -l


9361 /home/zhuw10/CL/SV/mocca-bench/data/truth_giab_padded.bed
16370 novoBreak_test/GIAB/nb_output/novoBreak.pass.bed
663


In [34]:
# /mnt/nfs/gigantor/ifs/DCEG/Projects/CoherentLogic/SV/mocca-new-callers/call_germline/TMP74218/isec.bed
head $ISEC_BED

1	184814727	184820784	HG3_PB_SVrefine2PBcRDovetail_565	DEL	6056	1	184814742	184820785	UID_208	DEL	+6042
1	247850454	247856508	HG4_PB_SVrefine2Falcon1Dovetail_917	DEL	6053	1	247850473	247856509	UID_248	DEL	+6035
1	226002813	226003141	HG2_Ill_scalpel_1190	DEL	327	1	226002816	226003142	UID_235	DEL	+325
1	241149855	241150181	HG3_Ill_GATKHCSBGrefine_897	DEL	325	1	241149872	241150182	UID_246	DEL	+309
1	25158686	25161509	HG3_Ill_GATKHCSBGrefine_157	DEL	2822	1	25158702	25161510	UID_41	DEL	+2807
1	198773748	198775414	HG3_PB_SVrefine2Falcon1Dovetail_658	DEL	1665	1	198773748	198775415	UID_218	DEL	+1666
1	104719846	104720164	HG3_10X_allpass_125	DEL	317	1	104719863	104720165	UID_77	DEL	+301
1	84517924	84524629	HG2_PB_SVrefine2Falcon1plusDovetail_434	DEL	6704	1	84517938	84524630	UID_68	DEL	+6691
1	58343210	58343535	HG2_Ill_MetaSV_107	DEL	324	1	58343222	58343536	UID_48	DEL	+313
1	26489811	26490138	HG4_Ill_GATKHCSBGrefine_190	DEL	326	1	26489822	26490139	UID_42	DEL	+316


## Check the one generate from the snakemake workflow

In [19]:
cd /home/zhuw10/CL/SV/mocca-new-callers

COMPUTE_VCF=novoBreak_test/snakemake_germline2/output/AJ_2x250_son/novoBreak.pass.flt.vcf
COMPUTE_BED=novoBreak_test/snakemake_germline2/output/AJ_2x250_son/novoBreak.pass.flt.bed
ISEC_BED=novoBreak_test/snakemake_germline2/output/AJ_2x250_son/novoBreak.pass.isec.bed
TRUTH_PAD=~/CL/SV/mocca-bench/data/truth_giab_padded.bed
INCLUDED_BED=~/CL/SV/mocca-bench/data/HG002_SVs_Tier1_noVDJorXorY_v0.6.2.bed

In [20]:
./scripts/novoBreak2bed.pl $COMPUTE_VCF > .tmp 
wc -l .tmp 
cut -f 5 .tmp | sort | uniq -c
bedtools intersect -a .tmp -b $INCLUDED_BED -f 1.0000000 -u > $COMPUTE_BED
cut -f 5 $COMPUTE_BED | sort | uniq -c
 

27409 .tmp
  16996 DEL
   5128 DUP
   5285 INV
   3235 DEL
      9 DUP
    233 INV


In [21]:
# .tmp is not reasonable. 
cat .tmp > $COMPUTE_BED

bedtools intersect -a $TRUTH_PAD -b $COMPUTE_BED -wa -wb -f 0.7 -r > $ISEC_BED

wc -l $TRUTH_PAD
wc -l $COMPUTE_BED 
cut -f 4 $ISEC_BED | sort -u |wc -l

9361 /home/zhuw10/CL/SV/mocca-bench/data/truth_giab_padded.bed
27409 novoBreak_test/snakemake_germline2/output/AJ_2x250_son/novoBreak.pass.flt.bed
1192


In [22]:
./scripts/novoBreak2bed.pl $COMPUTE_VCF | bedtools intersect -a - -b $INCLUDED_BED -f 1.0000000 -u > $COMPUTE_BED
cut -f 5 $COMPUTE_BED | sort | uniq -c

bedtools intersect -a $TRUTH_PAD -b $COMPUTE_BED -wa -wb -f 0.7 -r > $ISEC_BED

wc -l $TRUTH_PAD
wc -l $COMPUTE_BED 
cut -f 4 $ISEC_BED | sort -u |wc -l

   3235 DEL
      9 DUP
    233 INV
9361 /home/zhuw10/CL/SV/mocca-bench/data/truth_giab_padded.bed
3477 novoBreak_test/snakemake_germline2/output/AJ_2x250_son/novoBreak.pass.flt.bed
1192


In [23]:
head $ISEC_BED

cut -f 5 $ISEC_BED | sort | uniq -c 
cut -f 11 $ISEC_BED | sort | uniq -c 

1	205439854	205440024	HG2_10X_SVrefine210Xhap12_866	DEL	169	1	205439857	205440025	UID_622	DEL	+167
1	218785912	218786233	HG2_10X_allpass_181	DEL	320	1	218785929	218786234	UID_640	DEL	+304
1	27804111	27804441	HG2_Ill_scalpel_218	DEL	329	1	27804127	27804442	UID_113	DEL	+314
1	62390585	62390935	HG3_Ill_svaba_542	DEL	349	1	62390601	62390936	UID_166	DEL	+334
1	66135706	66136038	HG2_PB_SVrefine2PBcRplusDovetail_356	DEL	331	1	66135714	66136048	UID_173	DEL	+333
1	184814727	184820784	HG3_PB_SVrefine2PBcRDovetail_565	DEL	6056	1	184814742	184820785	UID_583	DEL	+6042
1	176478216	176478773	HG2_PB_SVrefine2Falcon2Bionano_216	DEL	556	1	176478231	176478774	UID_574	DEL	+542
1	240616989	240617314	HG4_Ill_manta_323	DEL	324	1	240617006	240617315	UID_729	DEL	+308
1	220502942	220503582	HG2_PB_pbsv_1179	DEL	639	1	220502944	220503583	UID_644	DEL	+638
1	180292988	180293313	HG3_10X_allpass_181	DEL	324	1	180293003	180293314	UID_579	DEL	+310
   1225 DEL
      1 INS
   1226 DEL
